In [1]:
import yaml
import os
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import matplotlib.pyplot as plt
from abc_env import ABC_env
from naive_agents import DoNothing, FixedPhi
from actor_physicist_agent import RL_phi_agent

import julia
jl = julia.Julia(sysimage="julia_img.so")
from julia import Main

### First make an open ai gym compliant version of the abc environment so we can train out of box RL algos

In [2]:

class ABC_env_gym(gym.Env):
    """
    Environment is 3D ABC environment!! 
    """
    def __init__(self,A,B,C,start_sep, beta, kappa, D, nu,seed=1):
        """ 
        - proper value for D should be derived from sampled lyapunov exponent
        """
        Main.include("abc_numerics.jl")

        super().__init__()
        self.A = A
        self.B = B
        self.C = C
        self.rng = np.random.default_rng(seed=seed)
        self.start_sep = start_sep
        self.sep_vec = self.rng.random(3) - 0.5
        self.sep_vec = self.sep_vec*start_sep/np.linalg.norm(self.sep_vec)
        self.passive = (self.rng.random(3) - 0.5) * 2 * np.pi
        self.active = self.active = self.passive + self.sep_vec
        self.reward=0
        self.deltaT=0.1 # environment step size
        self.time_step=0
        self.limit=10.
        self.kappa = kappa
        self.D = D
        self.nu = nu
        self.beta = beta
        
        
        self.action_space = spaces.Box(low=np.array([-5.]), high=np.array([5.]),dtype=np.float32)
        self.observation_space = spaces.Box(low=np.array([-100,-100,-100]),
                                            high=np.array([100,100,100],
                                            dtype=np.float32))

    # need to track active and passive not just separation vector

    def reset(self):
        self.sep_vec = self.rng.random(3) - 0.5
        self.sep_vec = self.sep_vec*self.start_sep/np.linalg.norm(self.sep_vec)
        self.passive = (self.rng.random(3) - 0.5) * 2 * np.pi
        self.active = self.active = self.passive + self.sep_vec
        self.reward=0
        self.time_step=0
        return np.array(self.getState(),dtype=np.float32)

    def step(self,action):
        # actions are just a selection of phi for thissteps = env.limit/env.deltaT * num_eps # amount of training steps allowed
        phi = action
        state_string = np.array2string(np.append(self.passive,self.active), separator=",")
        self.passive,self.active, penalty = Main.eval(f"envStep({self.A},{self.B},{self.C},{float(phi)}, {self.nu}, {self.kappa}, {self.beta}, {state_string},{self.deltaT})") 
        self.sep_vec = self.passive - self.active
        self.time_step += 1
        reward = -penalty
        return np.array(self.getState(),dtype=np.float32), reward, self.isOver(), {}


    def eval_step(self,phi):
        """
        Returns the baseline aproximate for the given phi value and the current state

        NOTE: In training you should always evaluate for a fixed phi not the phi the agent picks. 

        """

        dims = 3
        d_tilde = (dims+2) * (dims-1) * self.D
        #useful intermediate value used in the baseline mutiple times
        block = self.nu + 2*phi - d_tilde
        time_remaining = self.limit - self.getTime()
        a = ((self.beta + phi**2)*(1-np.exp(-time_remaining*block)))/block
        b_term1 = dims * self.kappa * (self.beta + phi**2) / (self.nu*(2*phi-d_tilde))
        b_term2 = 1 - np.exp(-self.nu*time_remaining) - self.nu * (1-np.exp(-time_remaining*block))/block
        b = b_term1 * b_term2
        return -(a*self.dist()**2 + b)




    def getState(self):
        steps = env.limit/env.deltaT * num_eps # amount of training steps allowed
        return self.sep_vec
    
    def isOver(self):
        return self.time_step * self.deltaT >= self.limit
        
    def dist(self):
        return np.linalg.norm(self.sep_vec)

    def getTime(self):
        return self.time_step * self.deltaT


In [3]:
# Load parameters for a given config file
config_file = "default_config_abc"

with open(f"config_files/{config_file}.yaml") as cf_file:
        config = yaml.safe_load( cf_file.read() )

# load params from file
dims=3 #abc flows is 3d
A = config["A"]
B = config["B"]
C = config["C"]
start_sep = config["start_sep"]
BETA = config["BETA"]
kappa = config["kappa"] 
D = config["D"]
NU = config["NU"] 
delta_t = config["delta_t"]
t_end = config["t_end"]
num_eps = config["num_eps"]

# intialize and test environment
env = ABC_env_gym(A,B,C,start_sep,BETA,kappa,D,NU)
env.limit = t_end
env.deltaT = delta_t
# It will check your custom environment and output additional warnings if needed
check_env(env)

/home/cfckoh/anaconda3/envs/stableBase3/lib/python3.11/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/cfckoh/anaconda3/envs/stableBase3/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:361: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [5]:
# train out of box A2C if saved model doesn't exists already

steps = env.limit/env.deltaT * num_eps # amount of training steps allowed
model_file="saved_models/stablebase_A2C.zip"
model_A2C = A2C("MlpPolicy", env, verbose=1)
if not os.path.isfile(model_file):
    model_A2C.learn(total_timesteps=steps)
    model_A2C.save("saved_models/stablebase_A2C")
else:
    model_A2C.load("stablebase_A2C.zip")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 200      |
|    ep_rew_mean        | -10.2    |
| time/                 |          |
|    fps                | 643      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.37    |
|    explained_variance | -13.8    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0463   |
|    std                | 0.953    |
|    value_loss         | 0.00212  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 200      |
|    ep_rew_mean        | -183     |
| time/                 |          |
|    fps                | 630      |
|    iterations   

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -3.49e+03 |
| time/                 |           |
|    fps                | 633       |
|    iterations         | 1300      |
|    time_elapsed       | 10        |
|    total_timesteps    | 6500      |
| train/                |           |
|    entropy_loss       | -1.24     |
|    explained_variance | -0.0087   |
|    learning_rate      | 0.0007    |
|    n_updates          | 1299      |
|    policy_loss        | -5.69     |
|    std                | 0.834     |
|    value_loss         | 41.1      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -3.21e+03 |
| time/                 |           |
|    fps                | 633       |
|    iterations         | 1400      |
|    time_elapsed       | 11        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -2.18e+03 |
| time/                 |           |
|    fps                | 645       |
|    iterations         | 2600      |
|    time_elapsed       | 20        |
|    total_timesteps    | 13000     |
| train/                |           |
|    entropy_loss       | -1.09     |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0007    |
|    n_updates          | 2599      |
|    policy_loss        | -1.71e+03 |
|    std                | 0.719     |
|    value_loss         | 2.03e+06  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.24e+05 |
| time/                 |           |
|    fps                | 645       |
|    iterations         | 2700      |
|    time_elapsed       | 20        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.49e+05 |
| time/                 |           |
|    fps                | 650       |
|    iterations         | 3900      |
|    time_elapsed       | 29        |
|    total_timesteps    | 19500     |
| train/                |           |
|    entropy_loss       | -1        |
|    explained_variance | -0.453    |
|    learning_rate      | 0.0007    |
|    n_updates          | 3899      |
|    policy_loss        | -4.34e-05 |
|    std                | 0.658     |
|    value_loss         | 2.3e-06   |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.44e+05 |
| time/                 |           |
|    fps                | 651       |
|    iterations         | 4000      |
|    time_elapsed       | 30        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -7.53e+05 |
| time/                 |           |
|    fps                | 650       |
|    iterations         | 5100      |
|    time_elapsed       | 39        |
|    total_timesteps    | 25500     |
| train/                |           |
|    entropy_loss       | -0.876    |
|    explained_variance | -23.9     |
|    learning_rate      | 0.0007    |
|    n_updates          | 5099      |
|    policy_loss        | 2.43      |
|    std                | 0.581     |
|    value_loss         | 8.17      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -7.53e+05 |
| time/                 |           |
|    fps                | 650       |
|    iterations         | 5200      |
|    time_elapsed       | 39        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 638       |
|    iterations         | 6300      |
|    time_elapsed       | 49        |
|    total_timesteps    | 31500     |
| train/                |           |
|    entropy_loss       | -0.788    |
|    explained_variance | -0.0334   |
|    learning_rate      | 0.0007    |
|    n_updates          | 6299      |
|    policy_loss        | -13       |
|    std                | 0.531     |
|    value_loss         | 115       |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 639       |
|    iterations         | 6400      |
|    time_elapsed       | 50        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 633       |
|    iterations         | 7500      |
|    time_elapsed       | 59        |
|    total_timesteps    | 37500     |
| train/                |           |
|    entropy_loss       | -0.657    |
|    explained_variance | -1.63     |
|    learning_rate      | 0.0007    |
|    n_updates          | 7499      |
|    policy_loss        | -0.00369  |
|    std                | 0.467     |
|    value_loss         | 6.98e-05  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 632       |
|    iterations         | 7600      |
|    time_elapsed       | 60        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 631       |
|    iterations         | 8700      |
|    time_elapsed       | 68        |
|    total_timesteps    | 43500     |
| train/                |           |
|    entropy_loss       | -0.576    |
|    explained_variance | 0.814     |
|    learning_rate      | 0.0007    |
|    n_updates          | 8699      |
|    policy_loss        | 0.00603   |
|    std                | 0.43      |
|    value_loss         | 0.000174  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 631       |
|    iterations         | 8800      |
|    time_elapsed       | 69        |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.52e+08 |
| time/                 |           |
|    fps                | 630       |
|    iterations         | 9900      |
|    time_elapsed       | 78        |
|    total_timesteps    | 49500     |
| train/                |           |
|    entropy_loss       | -0.533    |
|    explained_variance | -0.0635   |
|    learning_rate      | 0.0007    |
|    n_updates          | 9899      |
|    policy_loss        | -0.00449  |
|    std                | 0.412     |
|    value_loss         | 6.43e-05  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.37e+04 |
| time/                 |           |
|    fps                | 631       |
|    iterations         | 10000     |
|    time_elapsed       | 79        |
|    total_t

In [6]:
# train out of box PPO (a more sophisticated actor-critic algo)

steps = env.limit/env.deltaT * num_eps # amount of training steps allowed
model_file="saved_models/stablebase_PPO.zip"
model_PPO = PPO("MlpPolicy", env, verbose=1)
if not os.path.isfile(model_file):
    model_PPO.learn(total_timesteps=steps)
    model_PPO.save("saved_models/stablebase_PPO")
else:
    model_PPO.load("stablebase_PPO.zip")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -269     |
| time/              |          |
|    fps             | 892      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -280         |
| time/                   |              |
|    fps                  | 720          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0032697883 |
|    clip_fraction        | 0.0298       |
|    clip_range           | 0.2          |
|    en

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -425         |
| time/                   |              |
|    fps                  | 625          |
|    iterations           | 11           |
|    time_elapsed         | 36           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 9.980015e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.4         |
|    explained_variance   | -0.0408      |
|    learning_rate        | 0.0003       |
|    loss                 | 1.2e+03      |
|    n_updates            | 100          |
|    policy_gradient_loss | 0.000155     |
|    std                  | 0.98         |
|    value_loss           | 2.32e+03     |
------------------------------------------
------------------------------------------
| rollout/ 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -202         |
| time/                   |              |
|    fps                  | 598          |
|    iterations           | 20           |
|    time_elapsed         | 68           |
|    total_timesteps      | 40960        |
| train/                  |              |
|    approx_kl            | 0.0043099867 |
|    clip_fraction        | 0.031        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.167       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.321        |
|    n_updates            | 190          |
|    policy_gradient_loss | -0.00369     |
|    std                  | 0.945        |
|    value_loss           | 1.05         |
------------------------------------------
------------------------------------------
| rollout/ 

### Now compare different agents

In [7]:
compare_eps = 10
state = env.reset()
results = {}
for stable_model,name in [(model_A2C,"A2C"),(model_PPO,"PPO")]:
    cum_rew=np.zeros(int(env.limit/env.deltaT)+1)
    for ep in range(num_eps):
        step=0
        episode_rew=0
        while not env.isOver():
            action,_ = stable_model.predict(state)
            state, reward, _, _ = env.step(action)
            episode_rew += reward
            step+=1
            cum_rew[step] = episode_rew
        state = env.reset()
    results[name]=cum_rew

In [ ]:
fixed_phi = FixedPhi(0.9)
name="fixed_phi"
cum_rew=np.zeros(int(env.limit/env.deltaT)+1)
for ep in range(num_eps):
    step=0
    episode_rew=0
    while not env.isOver():
        action = fixed_phi.sample_action(state)
        state, reward, _, _ = env.step(action)
        episode_rew += reward
        step+=1
        cum_rew[step] = episode_rew
    state = env.reset()
    results[name]=cum_rew

In [ ]:
labels=[]
for name,rews in results.items():
    labels.append(name)
    plt.plot(np.arange(len(rews))/10,rews)
plt.legend(labels=labels)
plt.title("Avg Reward Over An Episode (ABC Flows)")
plt.xlabel("Time")
plt.ylabel("Avg Reward")
plt.ylim(-1,0.)
#plt.savefig(f"figs/compare_out_of_box.pdf", format="pdf", bbox_inches="tight")

In [ ]:
"""
TODO
- need to load the appropriate trained RL agent!!
- run stableBase agents
- run the other agents in the non-gym environment
- comparison plot between all agents

"""